In [ ]:
!pip install ultralytics

# Data Preprocessing: Split And Organize the datatset

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define old and new base directories
old_train_base = '/kaggle/input/haik-24-arabic-sign-language-image-classification/train/train/haik-image claassification'
new_base = '/kaggle/working/dataset'
new_train_base = os.path.join(new_base, 'train')
new_val_base = os.path.join(new_base, 'val')
new_test_base = os.path.join(new_base, 'test')

# Create new directories
os.makedirs(new_train_base, exist_ok=True)
os.makedirs(new_val_base, exist_ok=True)
os.makedirs(new_test_base, exist_ok=True)

# Split ratio
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

# Copy and split data
for class_name in os.listdir(old_train_base):
    old_class_path = os.path.join(old_train_base, class_name)
    if os.path.isdir(old_class_path):
        # Get list of all images in the class directory
        images = os.listdir(old_class_path)
        train_images, temp_images = train_test_split(images, test_size=(1 - train_ratio))
        val_images, test_images = train_test_split(temp_images, test_size=(test_ratio / (val_ratio + test_ratio)))

        # Define new paths for train, val, and test splits
        new_class_train_path = os.path.join(new_train_base, class_name)
        new_class_val_path = os.path.join(new_val_base, class_name)
        new_class_test_path = os.path.join(new_test_base, class_name)

        os.makedirs(new_class_train_path, exist_ok=True)
        os.makedirs(new_class_val_path, exist_ok=True)
        os.makedirs(new_class_test_path, exist_ok=True)

        # Copy images to new train directory
        for image_name in train_images:
            shutil.copy(os.path.join(old_class_path, image_name), os.path.join(new_class_train_path, image_name))

        # Copy images to new val directory
        for image_name in val_images:
            shutil.copy(os.path.join(old_class_path, image_name), os.path.join(new_class_val_path, image_name))

        # Copy images to new test directory
        for image_name in test_images:
            shutil.copy(os.path.join(old_class_path, image_name), os.path.join(new_class_test_path, image_name))


# Load the YOLO v8 classification model

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8n-cls.yaml")  # build a new model from YAML
model = YOLO("yolov8n-cls.pt")  # load a pretrained model
model = YOLO("yolov8n-cls.yaml").load("yolov8n-cls.pt")  # build from YAML and transfer weights

# Train the model
results = model.train(data="/kaggle/working/dataset", epochs=30, imgsz=64)

In [ ]:
# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.top1  # top1 accuracy
metrics.top5  # top5 accuracy

## predict the model with image from the test

In [ ]:
# Predict with the model
results = model("/kaggle/input/haik-24-arabic-sign-language-image-classification/test/test/124wqrf12 (10).JPG")  # predict on an image

In [ ]:
# Export the Model PyTorch Format ".pt"
model.export()

In [ ]:
# export the model ".onnx" format
model.export(format="onnx")

# **Testing the model with random samples from the test directory and ploting results**

In [ ]:
import os
import random
import matplotlib.pyplot as plt
from PIL import Image
from ultralytics import YOLO

# Define paths
test_base = '/kaggle/working/dataset/test'
model_path = '/kaggle/working/best.pt'

# Load the trained YOLO model
model = YOLO(model_path)

# Function to get a random image path from the test set
def get_random_image_path(test_base):
    class_name = random.choice(os.listdir(test_base))
    class_path = os.path.join(test_base, class_name)
    image_name = random.choice(os.listdir(class_path))
    return os.path.join(class_path, image_name), class_name

# Get class names
class_names = sorted(os.listdir(test_base))

# Number of images to test
num_images_to_test = 10

# Collect paths of random test images
image_paths = []
true_classes = []
for _ in range(num_images_to_test):
    image_path, true_class = get_random_image_path(test_base)
    image_paths.append(image_path)
    true_classes.append(true_class)

# Predict with the model
results = model(image_paths)  # Run batched inference on the list of images

# Plot results
plt.figure(figsize=(15, 10))
for i, result in enumerate(results):
    # Extract predicted class index and name
    predicted_class_index = result.probs.top1
    predicted_class = class_names[predicted_class_index]

    # Load the image for display
    img = Image.open(image_paths[i]).convert('RGB')

    plt.subplot(1, num_images_to_test, i + 1)
    plt.imshow(img)
    plt.title(f"True: {true_classes[i]}\nPred: {predicted_class}")
    plt.axis('off')

plt.tight_layout()
plt.show()
